# Measuring Exposure at Default


**Import the libraries that will be used**

In [1]:
import urllib.request as request
import json
from jsonschema import validate
import jsonschema
import pandas as pd
import numpy as np

Use json.loads to convert both the sample data and schema into JSON object

In [2]:
schema_path = request.urlopen("https://raw.githubusercontent.com/SuadeLabs/fire/master/v1-dev/derivative.json").read()
schema = json.loads(schema_path)
print (schema)

{'$schema': 'http://json-schema.org/draft-04/schema#', 'title': 'Derivative Schema', 'description': 'A derivative is a contract which derives its value from an underlying reference index, security or asset.', 'type': 'object', 'properties': {'id': {'description': 'The unique identifier for the derivative within the financial institution.', 'type': 'string'}, 'date': {'description': 'The observation or effective date for the data in this object. Formatted as YYYY-MM-DDTHH:MM:SSZ in accordance with ISO 8601.', 'type': 'string', 'format': 'date-time'}, 'deal_id': {'description': 'The unique identifier used by the financial institution for the deal to which this derivative belongs.', 'type': 'string'}, 'accounting_treatment': {'$ref': 'https://raw.githubusercontent.com/SuadeLabs/fire/master/v1-dev/common.json#/accounting_treatment'}, 'accrued_interest': {'description': 'The accrued interest since the last payment date and due at the next payment date. Monetary type represented as a natural

In [3]:
file_path = request.urlopen("https://raw.githubusercontent.com/mariankhoury/SuadeExercise/main/Data.json").read()
data = json.loads(file_path)
print (data)

{'name': 'Derivatives Data', 'date': '2017-01-17T00:00:00Z', 'data': [{'id': 'swap_1', 'date': '2017-01-17T00:00:00Z', 'asset_class': 'ir', 'currency_code': 'USD', 'end_date': '2019-01-17T00:00:00Z', 'mtm_dirty': -1500, 'notional_amount': 10000, 'payment_type': 'fixed', 'receive_type': 'floating', 'start_date': '2017-01-10T00:00:00Z', 'type': 'vanilla_swap', 'trade_date': '2017-01-10T00:00:00Z', 'value_date': '2017-01-16T00:00:00Z'}, {'id': 'swap_2', 'date': '2017-01-17T00:00:00Z', 'asset_class': 'ir', 'currency_code': 'GBP', 'end_date': '2019-01-17T00:00:00Z', 'mtm_dirty': 1400, 'notional_amount': 10500, 'payment_type': 'floating', 'receive_type': 'floating', 'start_date': '2017-01-10T00:00:00Z', 'type': 'vanilla_swap', 'trade_date': '2017-01-10T00:00:00Z', 'value_date': '2017-01-16T00:00:00Z'}]}


Use validate method is run to validate the data and that the data matches the schema

In [4]:
def validate(data):
    for item in data['data']:
        try:
            jsonschema.validate(item, schema)
        except Exception as e:
            raise e

In [5]:
validate(data)

Convert the sample data to a dataframe using pandas library

In [6]:
df = pd.DataFrame(data['data'])

In [7]:
df

,id,date,asset_class,currency_code,end_date,mtm_dirty,notional_amount,payment_type,receive_type,start_date,type,trade_date,value_date
0,swap_1,2017-01-17T00:00:00Z,ir,USD,2019-01-17T00:00:00Z,-1500,10000,fixed,floating,2017-01-10T00:00:00Z,vanilla_swap,2017-01-10T00:00:00Z,2017-01-16T00:00:00Z
1,swap_2,2017-01-17T00:00:00Z,ir,GBP,2019-01-17T00:00:00Z,1400,10500,floating,floating,2017-01-10T00:00:00Z,vanilla_swap,2017-01-10T00:00:00Z,2017-01-16T00:00:00Z


**To calculate the adjusted notional:**

First, use pandas to parse contract dates into datetime objects

In [8]:
start = pd.to_datetime(df['start_date'].values)
end = pd.to_datetime(df['end_date'].values)
current = pd.to_datetime(df['date'].values)

Calculate both S and E and add their columns to the dataframe 

In [9]:
df['E'] = (end - start).days / 365

In [10]:
S = (start-current).days/365
df['S'] = np.where(
    S >0, S, np.where(
    S <  0, 0,-1)) 
#where argument in order to return 0 for any negative numbers

To calculate the **adjusted notional**, calculate the **supervisory duration** first, then multiply by the notional amount. The supervisory duration calculation is referncing **paragraph 157** of http://www.bis.org/publ/bcbs279.pdf

In [12]:
sup_duration = (np.exp(-0.05 * df['S']) - np.exp(-0.05 * df['E'])) / 0.05
sup_duration

0    1.920596
1    1.920596
dtype: float64

**Adjusted notional** calculating references **paragraph 157** of http://www.bis.org/publ/bcbs279.pdf

In [13]:
df['adj_notional']= (df['notional_amount'] * sup_duration).values
#add the adjusted notional to the dataframe

Calculate **replacement cost** as outlined in **paragraph 136** of http://www.bis.org/publ/bcbs279.pdf

In [14]:
def replacement_cost():
    replacement_cost = np.sum(df['mtm_dirty'].values)
    return max(replacement_cost, 0)

In [16]:
replacement_cost()

0

In [15]:
derivatives_values = np.sum(df['mtm_dirty'].values)

In [17]:
derivatives_values

-100

Calculate the **supervisory delta adjustments** as in **paragraph 159** of http://www.bis.org/publ/bcbs279.pdf

In [18]:
def floating_fixed(x):
    if x == 'fixed':
        return -1
    else:
        return 1  

In [19]:
df['supervisory_delta']= df['receive_type'].apply(lambda x: floating_fixed(x)).values

Calculate the **maturity factor** by first calculating the **maturities of the derivatives**, as outlined below. This is referenced in **paragraph 164** of http://www.bis.org/publ/bcbs279.pdf

In [20]:
def swap_maturity():
    end =pd.to_datetime(df['end_date'].values)
    current = pd.to_datetime(df['date'].values)
    swap_maturity = (end - current).days/365
    return swap_maturity.values

In [21]:
swap_maturity = swap_maturity()

In [22]:
def maturity_factor():
    mf = list(map(lambda x: min(x,1), swap_maturity))
    return np.sqrt(mf) 

The **effective notional** is calculated based on the derived parameters and is then added to the dataframe, referencing **paragraph 167** of http://www.bis.org/publ/bcbs279.pdf

In [23]:
df['effective_notional']= df['adj_notional'] * df['supervisory_delta'] * maturity_factor()


In [24]:
df

,id,date,asset_class,currency_code,end_date,mtm_dirty,notional_amount,payment_type,receive_type,start_date,type,trade_date,value_date,E,S,adj_notional,supervisory_delta,effective_notional
0,swap_1,2017-01-17T00:00:00Z,ir,USD,2019-01-17T00:00:00Z,-1500,10000,fixed,floating,2017-01-10T00:00:00Z,vanilla_swap,2017-01-10T00:00:00Z,2017-01-16T00:00:00Z,2.019178,0.0,19205.963684,1,19205.963684
1,swap_2,2017-01-17T00:00:00Z,ir,GBP,2019-01-17T00:00:00Z,1400,10500,floating,floating,2017-01-10T00:00:00Z,vanilla_swap,2017-01-10T00:00:00Z,2017-01-16T00:00:00Z,2.019178,0.0,20166.261868,1,20166.261868


The **Add-on** is calculated below, according to **paragraphs 166-169** of http://www.bis.org/publ/bcbs279.pdf

In [25]:
def addon():
    addon = np.sum(0.005*df['effective_notional'])
    return addon

In [26]:
addon()

196.86112775680456

In [27]:
addon = addon()

Finally, in order to calculate the **Exposure at Default**, the multiplier is calculated as outlined in **paragraph 148** of http://www.bis.org/publ/bcbs279.pdf

In [28]:
floor = 0.0
derivatives_values= np.sum(df['mtm_dirty'].values)
numerator = derivatives_values
denominator = 2 * (1 - floor) * addon
exponent = np.exp(numerator / denominator)


In [29]:
def multiplier():
    multiplier = floor + (1 - floor) * exponent
    multiplier = min(1, multiplier)
    return multiplier

In [30]:
multiplier()

0.7757025451513244

In [31]:
multiplier = multiplier()

Now, we have all the variables to calculate the **Exposure at Default** as outlined in **paragraph 128** of http://www.bis.org/publ/bcbs279.pdf. The calculation is:

In [32]:
rc = replacement_cost()
alpha = 1.4
exposure_at_default = alpha * (rc + (multiplier * addon))

In [33]:
exposure_at_default

213.78794897923868